In [50]:
import geonamescache
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

gc = geonamescache.GeonamesCache()
countries_re = "|".join(list(gc.get_countries_by_names().keys()))
comp_country_re = re.compile(r"\b" + countries_re + r"\b")

cities = [value.get("name") for key, value in gc.get_cities().items()]

# Dataframe columns
df_headline = []
df_country = []
df_city = []

file = open("data/headlines.txt", 'r')
for line in file.readlines():
    df_headline.append(line.rstrip())
    result = comp_country_re.search(unidecode(line.rstrip()))
    if result is not None:
        df_country.append(result.group(0))
    else:
        df_country.append(np.NaN)
        
    headlinewords = re.findall(r"([A-Z]\w+)\s?", line)
    found_cities = []
    
    while(len(headlinewords) > 0):
        wordSubset = headlinewords[:]
        
        while(len(wordSubset) > 0):
            result = (" ".join(wordSubset)).strip()
            if result in cities:
                if not result in df_city:
                    found_cities.append(result)
            wordSubset.pop()
        headlinewords.pop(0)
    found_cities = list(filter( lambda n: len([city for city in found_cities if n in city]) ==1, found_cities))
    df_city.append(found_cities[0] if len(found_cities) > 0 else np.NaN)

file.close()

df = pd.DataFrame({'headline': df_headline, 'country': df_country, 'city': df_city})

print(df)
print(df.head(50))



                                              headline country           city
0                             Zika Outbreak Hits Miami     NaN          Miami
1                      Could Zika Reach New York City?     NaN  New York City
2                    First Case of Zika in Miami Beach     NaN    Miami Beach
3              Mystery Virus Spreads in Recife, Brazil  Brazil         Recife
4              Dallas man comes down with case of Zika     NaN         Dallas
..                                                 ...     ...            ...
645  Rumors about Rabies spreading in Jerusalem hav...     NaN      Jerusalem
646              More Zika patients reported in Indang     NaN         Indang
647  Suva authorities confirmed the spread of Rotav...     NaN           Suva
648         More Zika patients reported in Bella Vista     NaN    Bella Vista
649                     Zika Outbreak in Wichita Falls     NaN  Wichita Falls

[650 rows x 3 columns]
                                        